In [ ]:
from pxr import Usd, Sdf

# Create a new stage
new_stage = Usd.Stage.CreateNew('referencedScene.usd')

# Define a prim in the new stage to host the reference
ref_prim = new_stage.DefinePrim('/world/asset')

# Get the references for the newly created prim
references = ref_prim.GetReferences()

# Add a reference to the existing USD file
references.AddReference(assetPath='./assets/g1_23dof_simple.usd')

# Save the stage
new_stage.Save()

print("Successfully referenced myAsset.usda into referencedScene.usd")

Successfully referenced myAsset.usda into referencedScene.usd


In [3]:
from pxr import Usd, Sdf

# Create a new, empty stage
new_stage = Usd.Stage.CreateNew('newScene.usd')

# Get the root layer of the new stage
root_layer = new_stage.GetRootLayer()

# Add the existing USD file as a sublayer
root_layer.subLayerPaths.append('./assets/g1_23dof_simple.usd')

# Save the new stage
new_stage.Save()

print("Successfully added existingScene.usda as a sublayer to newScene.usd")

Successfully added existingScene.usda as a sublayer to newScene.usd


In [15]:
from pxr import Usd, UsdGeom, Gf, Sdf

# Define the input asset path
asset_path = './assets/g1_23dof_simple.usd'

# Create a new, empty stage in memory
stage = Usd.Stage.CreateInMemory()

# --- Create the first (original) instance ---
# Define a prim for the first robot
robot1_prim = stage.DefinePrim('/World/Robot_Original')

# Add a reference to the asset under this prim
robot1_prim.GetReferences().AddReference(assetPath=asset_path)


# --- Create the second (modified) instance ---
# Define a prim for the second robot
robot2_prim = stage.DefinePrim('/World/Robot_Modified')

# Add a reference to the same asset
robot2_prim.GetReferences().AddReference(assetPath=asset_path)

# Move the second robot 100 units (1m) to the right on the X axis
# xformable = UsdGeom.Xformable(robot2_prim)
# xformable.AddTranslateOp().Set(Gf.Vec3d(100, 0, 0))
xformable = UsdGeom.Xformable(robot2_prim)
xformable.ClearXformOpOrder() 
# Now, add the translate op. This will be the ONLY op on this prim.
xformable.AddTranslateOp().Set(Gf.Vec3d(0, 1, 0))

# --- Modify the knee joint on the second robot ---
# Construct the path to the right knee joint prim on the second robot
knee_joint_path = Sdf.Path('/World/Robot_Modified/joints/right_knee_joint')


# # Get the joint prim from the stage
knee_joint_prim = stage.GetPrimAtPath(knee_joint_path)



if knee_joint_prim:
    # Get the 'state:angular:physics:position' attribute, which controls the joint's rotation.
    # This is the correct attribute for setting the joint's current angle.
    joint_pos_attr = knee_joint_prim.GetAttribute('state:angular:physics:position')

    if joint_pos_attr:
        # Set the knee bend to -90.0 degrees. This will bend the knee.
        joint_pos_attr.Set(-90.0)
        print(f"Successfully set 'state:angular:physics:position' for {knee_joint_path}")
    else:
        print(f"Warning: Could not find 'state:angular:physics:position' attribute on prim {knee_joint_path}")

else:
    print(f"Error: Could not find prim at path {knee_joint_path}")


# Save the resulting stage to a file
stage.GetRootLayer().Export('side_by_side_comparison_corrected.usda')

print("\nSuccessfully created 'side_by_side_comparison_corrected.usda' with two robots.")
print("The robot at /World/Robot_Modified has a bent right knee using the correct state attribute.")

Successfully set 'state:angular:physics:position' for /World/Robot_Modified/joints/right_knee_joint

Successfully created 'side_by_side_comparison_corrected.usda' with two robots.
The robot at /World/Robot_Modified has a bent right knee using the correct state attribute.
